In [ ]:
import os
import json
import csv
import random
from openai import OpenAI


Processing LogicBenchFiles/Eval .json Files/BQA\EVAL_DysjunctivSyl(BQA).json...
Saved: LogicBenchFiles/Eval .json Files/BQA\EVAL_DysjunctivSyl(BQA)_o3_completed.csv
Processing LogicBenchFiles/Eval .json Files/BQA\EVAL_HypotheticalSyl(BQA).json...
Saved: LogicBenchFiles/Eval .json Files/BQA\EVAL_HypotheticalSyl(BQA)_o3_completed.csv
Processing LogicBenchFiles/Eval .json Files/BQA\EVAL_ModusPones(BQA).json...
Saved: LogicBenchFiles/Eval .json Files/BQA\EVAL_ModusPones(BQA)_o3_completed.csv
Processing LogicBenchFiles/Eval .json Files/BQA\EVAL_ModusTollens(BQA).json...
Saved: LogicBenchFiles/Eval .json Files/BQA\EVAL_ModusTollens(BQA)_o3_completed.csv
Processing LogicBenchFiles/Eval .json Files/MCQA\EVAL_DisjunctiveSyl(MCQA).json...
Saved: LogicBenchFiles/Eval .json Files/MCQA\EVAL_DisjunctiveSyl(MCQA)_o3_completed.csv
Processing LogicBenchFiles/Eval .json Files/MCQA\EVAL_HypotheticalSyl(MCQA).json...
Saved: LogicBenchFiles/Eval .json Files/MCQA\EVAL_HypotheticalSyl(MCQA)_o3_completed.csv


In [ ]:


# initialize client once
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# model name for output files
model_name = "o3"

# folders
BQA_FOLDER = "LogicBenchFiles/Eval .json Files/BQA"
MCQA_FOLDER = "LogicBenchFiles/Eval .json Files/MCQA"

# for consistent end prompts
ENDPROMPT_BQA = "Only answer 'yes' or 'no' do not provide any additional characters like punctuation or explanation just yes or no thats it.:"
ENDPROMPT_MCQA = "Do not provide any additional characters like punctuation or explanation just the choice thats it (ex. Choice_1, Choice_2, Choice_3, Choice_4).:"


In [ ]:

# function to process BQA files
def process_bqa(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)

    rows = []
    for current_q in data["samples"]:
        for qa1 in current_q["qa_pairs"]:
            prompt = current_q["context"] + "\n" + qa1["question"] + "\n" + ENDPROMPT_BQA

            response = client.responses.create(
                model= model_name,
                input=prompt,
                store=False,
            )
            chat_gpt_answer = response.output_text.lower().strip()

            rows.append({
                "Prompt": prompt,
                "Context": current_q["context"],
                "Question": qa1["question"],
                "Endprompt": ENDPROMPT_BQA,
                "Correct answer": qa1["answer"],
                "ChatGPT answer": chat_gpt_answer
            })
    return rows


# function to process MCQA files
def process_mcqa(file_path):
    with open(file_path, "r") as file:
        data = json.load(file)

    rows = []
    for current_q in data["samples"]:
        prompt = current_q["context"] + "\n" + current_q["question"] + " Select the best answer from the choices below:\n\n"
        for key, value in current_q["choices"].items():
            prompt += f"{key}: {value}\n"
        prompt += "\n" + ENDPROMPT_MCQA

        response = client.responses.create(
            model= model_name,
            input=prompt,
            store=False,
        )
        chat_gpt_answer = response.output_text.lower().strip()

        rows.append({
            "Prompt": prompt,
            "Context": current_q["context"],
            "Question": current_q["question"],
            "Endprompt": ENDPROMPT_MCQA,
            "Correct answer": current_q["answer"],
            "ChatGPT answer": chat_gpt_answer
        })
    return rows


def write_to_csv(file_path, rows):
    # Extract just the filename, replace extension with _completed.csv
    base_name = os.path.basename(file_path).replace(".json", f"_{model_name}_completed.csv")
    # Save into the same folder as the input file
    folder = os.path.dirname(file_path)
    csv_file = os.path.join(folder, base_name)

    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["Prompt", "Context", "Question", "Endprompt", "Correct answer", "ChatGPT answer"])
        writer.writeheader()
        writer.writerows(rows)
    print(f"Saved: {csv_file}")


# main loop
for folder, processor in [(BQA_FOLDER, process_bqa), (MCQA_FOLDER, process_mcqa)]:
    for filename in os.listdir(folder):
        if filename.endswith(".json"):
            filepath = os.path.join(folder, filename)
            print(f"Processing {filepath}...")
            rows = processor(filepath)
            write_to_csv(filepath, rows)

In [1]:
import os
from openai import OpenAI
import random
import csv
import json

In [3]:
#model_name = "gpt-5"

client = OpenAI(
    api_key= os.getenv("OPENAI_API_KEY")
    )


response = client.responses.create(
  model= "gpt-4o-mini",
  input="Hello ChatGPT, What model are you?",
  store=True,
)

print(response.output_text);


Hello! I'm based on OpenAI's GPT-3 model. How can I assist you today?


In [4]:
with open('LogicBenchFiles/Eval .json Files/BQA/EVAL_ModusPones(BQA).json') as file :
    questions = file.read()

questions = json.loads(questions)
questions

{'type': 'first_order_logic',
 'axiom': 'modus_ponens',
 'samples': [{'id': 1,
   'context': 'If someone is extremely tired, then they will seek some rest and relaxation. Today, Jack finds himself utterly exhausted.',
   'qa_pairs': [{'question': 'Does this entail that he will take rest?',
     'answer': 'yes'},
    {'question': "Does this entail that he won't take rest?",
     'answer': 'no'}]},
  {'id': 2,
   'context': 'If a person experiences a throbbing sensation in their head, it is likely that they will seek relief by consuming medication intended to alleviate this discomfort. In this situation, Sarah finds herself with a pounding headache.',
   'qa_pairs': [{'question': 'Does this mean that she will take medicine?',
     'answer': 'yes'},
    {'question': "Does this mean that she won't take medicine?",
     'answer': 'no'}]},
  {'id': 3,
   'context': 'If someone experiences sadness, they will often shed tears. In this particular case, Sara is feeling sorrowful.',
   'qa_pairs'

In [5]:
with open('LogicBenchFiles/Eval .json Files/MCQA/EVAL_ModusPones(MCQA).json') as file :
    questions2 = file.read()

questions2 = json.loads(questions2)
questions2

{'type': 'first_order_logic',
 'axiom': 'modus_ponens',
 'samples': [{'id': 1,
   'context': 'If someone is extremely tired, then they will seek some rest and relaxation. today, jack finds himself utterly exhausted.',
   'question': 'Considering the provided context, what conclusion would be deemed most suitable?',
   'choices': {'choice_1': 'Emily will watch a movie',
    'choice_2': 'Sarah will go for a run',
    'choice_3': 'Jake will clean his house',
    'choice_4': 'He will take rest'},
   'answer': 'choice_4'},
  {'id': 2,
   'context': 'If a person experiences a throbbing sensation in their head, it is likely that they will seek relief by consuming medication intended to alleviate this discomfort. in this situation, sarah finds herself with a pounding headache.',
   'question': 'Considering the provided context, what conclusion would be deemed most suitable?',
   'choices': {'choice_1': 'Tom will offer Sarah some medicine.',
    'choice_2': 'She will take medicine.',
    'choic

In [6]:
q3 = questions2['samples']
len(q3)
print(q3[0].keys())

print(q3[0]['answer'])
print(q3[0]['choices'])



dict_keys(['id', 'context', 'question', 'choices', 'answer'])
choice_4
{'choice_1': 'Emily will watch a movie', 'choice_2': 'Sarah will go for a run', 'choice_3': 'Jake will clean his house', 'choice_4': 'He will take rest'}


In [7]:
q = questions['samples']
len(q)

20

In [8]:
print(q[0].keys())

print(q[0]['context'])

q[5]['qa_pairs']

dict_keys(['id', 'context', 'qa_pairs'])
If someone is extremely tired, then they will seek some rest and relaxation. Today, Jack finds himself utterly exhausted.


[{'question': 'Does this entail that she will plan a trip?', 'answer': 'yes'},
 {'question': "Does this entail that she won't plan a trip?", 'answer': 'no'}]

Each prompt:
- include context ('context')
- include question ('q_pairs')
- compare to answer ('answer')

In [9]:
q[5]['qa_pairs'][0]['answer']

'yes'

In [12]:


for current_q in q:
        
    for qa1 in current_q['qa_pairs']:
        prompt = '' 
        prompt += current_q['context'] + '\n' + qa1['question'] + '\n' + "Only answer 'yes' or 'no' do not provide any additonal characters like punctuation or explantation just yes or no thats it.:"

        client = OpenAI(
        api_key= os.getenv("OPENAI_API_KEY")
        )   


        response = client.responses.create(
            model= "gpt-4o-mini",
            input= prompt,
            store=False,
        )
        chat_gpt_answer = (response.output_text);




        print(prompt)
        num = random.random()
        chat_gpt = ' '
        if num > 0.6:
            chat_gpt = 'no'
        elif num > 0.2:
            chat_gpt = 'yes'
        print('ChatGPT answer:', chat_gpt_answer.lower().strip())
        print('answer:', qa1['answer'])
        print()



If someone is extremely tired, then they will seek some rest and relaxation. Today, Jack finds himself utterly exhausted.
Does this entail that he will take rest?
Only answer 'yes' or 'no' do not provide any additonal characters like punctuation or explantation just yes or no thats it.:
ChatGPT answer: yes
answer: yes

If someone is extremely tired, then they will seek some rest and relaxation. Today, Jack finds himself utterly exhausted.
Does this entail that he won't take rest?
Only answer 'yes' or 'no' do not provide any additonal characters like punctuation or explantation just yes or no thats it.:
ChatGPT answer: no
answer: no

If a person experiences a throbbing sensation in their head, it is likely that they will seek relief by consuming medication intended to alleviate this discomfort. In this situation, Sarah finds herself with a pounding headache.
Does this mean that she will take medicine?
Only answer 'yes' or 'no' do not provide any additonal characters like punctuation or 

In [11]:
for current_q2 in q3:
        prompt1 = '' 
        prompt1 += current_q2['context'] + '\n' + current_q2['question'] + ' Select the best answer from the choices below:\n\n' 
        
        for key, value in current_q2['choices'].items():
            prompt1 += f"{key}: {value}\n"


        prompt1 += "\nDo not provide any additonal characters like punctuation or explantation just the choice thats it(ex. Choice_1, Choice_2, Choice_3, Choice_4).:"

    
        client = OpenAI(
        api_key= os.getenv("OPENAI_API_KEY")
        )   


        response = client.responses.create(
            model= "gpt-4o-mini",
            input= prompt1,
            store=False,
        )
        chat_gpt_answer2 = (response.output_text);
        


        
        print(prompt1)
        num = random.random()
        chat_gpt2 = ' '
        if num > 0.6:
            chat_gpt2 = 'choice 1'
        elif num > 0.2:
            chat_gpt2 = 'choice 2'
        print('ChatGPT answer:', chat_gpt_answer2.lower().strip())
        print('answer:', current_q2['answer'])
        print()

If someone is extremely tired, then they will seek some rest and relaxation. today, jack finds himself utterly exhausted.
Considering the provided context, what conclusion would be deemed most suitable? Select the best answer from the choices below:

choice_1: Emily will watch a movie
choice_2: Sarah will go for a run
choice_3: Jake will clean his house
choice_4: He will take rest

Do not provide any additonal characters like punctuation or explantation just the choice thats it(ex. Choice_1, Choice_2, Choice_3, Choice_4).:
ChatGPT answer: choice_4
answer: choice_4

If a person experiences a throbbing sensation in their head, it is likely that they will seek relief by consuming medication intended to alleviate this discomfort. in this situation, sarah finds herself with a pounding headache.
Considering the provided context, what conclusion would be deemed most suitable? Select the best answer from the choices below:

choice_1: Tom will offer Sarah some medicine.
choice_2: She will take 